# Enumerate Facet inequalities for a given number of inputs and binary outputs
Here I want to implement an algorithm, to find all facet inequalities for a given number of inputs and binary outputs.
This follows the construction of *Algorithm 1* in the paper [*Bell inequalities from no-signalling distribution*, *Cope & Colbeck (2019)*](https://arxiv.org/abs/1812.10017).

First we set the number of inputs and outputs and obtain the extremal points for the specific NS polytope.

In [6]:
from utils import extremal_ns_binary_vertices, get_deterministic_behaviors
import numpy as np
# set inputs / outputs
inputs_a = range(3)
inputs_b = range(2)
outputs = range(2)

# get extremal points
extremals = extremal_ns_binary_vertices(inputs_a, inputs_b, outputs)

# get deterministic points
dets = get_deterministic_behaviors(inputs_a, inputs_b, outputs)

In [7]:
from utils import find_local_weight, facet_inequality_check, check_diff_repr_same_ineq
from itertools import product
epsilons = np.linspace(0,1/2, num=2)

# list of all facets
facets = []
for epsilon in epsilons:
    # iterate through extremals
    for e in extremals:
        # find the local weight for the extremal behavior
        opt, bell_expression = find_local_weight(e, dets, method='simplex')
        # check that there is no local part, as it is an extremal point
        assert np.abs(bell_expression @ e) < 1e-6
        # find all equalizing local behaviors for the inequality bell_expression @ d >= 1
        is_facet, bell_expression, eq_dets = facet_inequality_check(dets, bell_expression, len(inputs_a), len(inputs_b), len(outputs))
        # check if other representation is already in facets
        check = [check_diff_repr_same_ineq(bell_expression,f,dets) for f in facets]
        if True in check:
            continue
        # if it's a facet append to all facets
        # TODO: Check for relabelling
        if is_facet:
            facets.append(bell_expression)
        # define new behaviors based on epsilon
        for j,k in product(range(eq_dets.shape[0]),repeat=2):
            # need distinct j,k
            if j == k:
                continue
            # form new probability distribution
            e_new = (1-3*epsilon/2)*e + epsilon*eq_dets[j] + epsilon / 2 * eq_dets[k]
            # do the facet check again
            opt, bell_expression = find_local_weight(e_new, dets,method='simplex')
            is_facet, bell_expression, eq_dets_new = facet_inequality_check(dets, bell_expression, len(inputs_a), len(inputs_b), len(outputs))
            check = [check_diff_repr_same_ineq(bell_expression,f,dets) for f in facets]
            if True in check:
                continue
            if is_facet:
                facets.append(bell_expression)
print('Number of facets found: {}'.format(len(facets)))

Number of facets found: 3


Now we have found some facets for the specific case.
However we still have to check for similar Bell inequalities under relabelling.
